In [1]:
import os
os.environ['PGHOST']="aaz.chicagobooth.edu"
os.environ['PGDATABASE']="postgres"

In [2]:
from sqlalchemy import create_engine
from pandas.io.sql import read_sql
from run_NER import conn_string
print(conn_string)

postgresql://aaz.chicagobooth.edu/postgres


In [3]:
file_name = '1215386_T'
engine = create_engine(conn_string)
# It may be better to explicitly create the table elsewhere.
# Checking like this might be slower.

sql = """
    SELECT file_name, last_update, speaker_number, speaker_text
    FROM (
        SELECT file_name, last_update, speaker_number, speaker_text
        FROM streetevents.speaker_data
        WHERE context = 'qa') AS s
    JOIN (
        SELECT file_name, last_update,
            unnest(streetevents.qa_pairs.question_nums) AS speaker_number
        FROM streetevents.qa_pairs) AS q
    USING (file_name, last_update, speaker_number)
    WHERE file_name='%s'
""" % (file_name)

df = read_sql(sql, engine)
engine.dispose()

In [4]:
df

,file_name,last_update,speaker_number,speaker_text
0,1215386_T,2006-03-09 21:21:37,24,Okay. Does probably not mean that this will ha...
1,1215386_T,2006-03-09 21:21:37,4,"Then on the first quarter, maybe, can you give..."
2,1215386_T,2006-03-09 21:21:37,22,I have a question concerning Italy. I'm still ...
3,1215386_T,2006-03-09 21:21:37,8,Yes. And assuming that it is slightly positive...
4,1215386_T,2006-03-09 21:21:37,2,First question regarding the competition in Ge...
5,1215386_T,2006-03-09 21:21:37,17,I wondered if you could give us first of all s...
6,1215386_T,2006-03-09 21:21:37,10,"And maybe it's a little bit too early, but -- ..."
7,1215386_T,2006-03-09 21:21:37,12,One last question. You mentioned the Swiss mar...
8,1215386_T,2006-03-09 21:21:37,14,Maybe one very last question. You just reached...
9,1215386_T,2006-03-09 21:21:37,26,But I can assume that given what happened in e...


In [5]:
from findner import findner, tagger_init, get_positions, flatten

In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize
text_array = df['speaker_text']
sents = [sent_tokenize(speaker_text) for speaker_text in text_array]
sentence_lists = [word_tokenize(s) for s in flatten(sents)]

In [7]:
print([len(s) for s in sentence_lists[:5]])
print(get_positions(sentence_lists[:5]))

[2, 21, 32, 7, 16]
[[0, 2], [2, 23], [23, 55], [55, 62], [62, 78]]


In [8]:
# Confirm that old, SLOW function still works
tagger_init()
df['speaker_text'][:5].map(findner)

0            {"DATE": ["first", "quarter", "onwards"]}
1                       {"DATE": ["first", "quarter"]}
2    {"LOCATION": ["Italy", "Italy"], "ORGANIZATION...
3                                                   {}
4       {"LOCATION": ["Germany"], "DATE": ["January"]}
Name: speaker_text, dtype: object